In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


In [ ]:
os.chdir('/content/gdrive/My Drive/주분')

In [ ]:
train_nonanswer = pd.read_csv("train_nonanswer_jo_sum_final.csv")
test_nonanswer = pd.read_csv("test_nonanswer_jo_sum.csv")

In [ ]:
##------------- 'party','user_id' drop---------------------##

nonanswer_x = train_nonanswer.drop(['Party','USER_ID'],axis = 1)
nonanswer_y = train_nonanswer['Party']

In [ ]:
##------------- X 변수들 dummy화 해주기---------------------##
nonanswer_x_d = pd.get_dummies(nonanswer_x)

In [ ]:
##------------- test set dummy화 & user_id drop 해주기---------------------##
test_nonanswer_d=pd.get_dummies(test_nonanswer)
test_nonanswer_d = test_nonanswer_d.drop(['USER_ID'],axis = 1)

In [ ]:
from lightgbm import LGBMClassifier as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_seleciont import KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
best_param= {'eval_metric' : 'accuracy',
              'early_stopping_rounds' : 20, 
              'verbose' :-1,
              'bagging_fraction':0.0002, #
         'colsample_bytree':0, #
         'feature_fraction':0.7, #
         'learning_rate':0.003, #
         'max_depth':-1, #
         'min_child_weight':40, #
         'n_estimators':900, #
         'num_leaves': 30, #
         'reg_alph':0, #
         'reg_lambda':40, #
         'subsample':1, #
         'max_bin':800 #
}


In [ ]:
train_X, val_X, train_y, val_y = train_test_split(nonanswer_x_d, nonanswer_y, test_size = 0.15, train_size = 0.85, random_state = 123)
      
# lgb()
model_grid =lgb(**best_param,random_seed = 123)

# 모델 학습
model_grid.fit(train_X, train_y, eval_set=(val_X, val_y))

In [ ]:
pred_y = model_grid.predict(test_nonanswer_d)

In [ ]:
user_id = list(test_nonanswer['USER_ID'])

submission_grid=pd.DataFrame({'USER_ID':user_id, 'Predictions':pred_y})

In [ ]:
submission_grid.to_csv('submission_grid.csv',index=False)

In [ ]:
!pip install shap

In [ ]:
# Shpley Value 패키지

import shap

# Shpley Value 보기

explainer = shap.TreeExplainer(model_grid)
shap_values = explainer.shap_values(test_nonanswer_d)

shap.initjs()
shap.summary_plot(shap_values[1],test_nonanswer_d,plot_type = 'bar')